<a href="https://colab.research.google.com/github/h3it-dias/FastCamp_Agent_AI_CEIA/blob/main/Pydantic/Pydantic_pratica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydantic
!pip install 'pydantic[email]'

Importação dos modulos que serão necessários.

In [ ]:
from datetime import datetime
from typing import Optional
from uuid import uuid4, UUID

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

import enum
import hashlib
import re
from typing import Any, Self

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

Modelos de Cpf e nome que serão aceitos, codificados em regex.

In [ ]:
VALID_CPF_REGEX = re.compile(r"^\d{3}\.?\d{3}\.?\d{3}-?\d{2}$")
VALID_NOME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

Criação da classe Role, que é usada para instaciar as funções dos respectivos objetos(usuários), sendo as funções: Espectador, organizador e palestrante. Também foi criado a classe Usuario, que herda BaseModel para criar um modelo de dados que serão aceitos no sistema. Além disso, nessa mesma classe, temos as validações personalizadas para o contexto (Sistema de incrição para eventos), sendo elas: validação se o cpf e o nome seguem os padrões determinados anteriormente, e se role (função do usuário no evento) está correta com as funções que são aceitas. Em adição, temos a serialização do objeto em um json.

In [ ]:
class Role(enum.IntFlag):
  Espectador = 1
  Organizador = 2
  Palestrante = 4

class Usuario(BaseModel):
  model_config = {"extra": "forbid"}
  id: UUID = Field(default_factory=uuid4)
  nome: str = Field(examples="Heitor")
  email: EmailStr = Field(examples="exemplo@gmail.com", description="O email do usuário", frozen=True)
  cpf: SecretStr = Field(
      description="O Cpf do usário",
      examples="123.456.789.12",
      exclude=True,
  )
  role: Role = Field(
      description="A função do usuário no evento",
      examples="Palestrante",
      validate_default=True,
  )

  @field_validator("cpf")
  @classmethod
  def valida_cpf(cls, c: SecretStr) -> SecretStr:
    if not VALID_CPF_REGEX.match(c.get_secret_value()):
      raise ValueError("CPF inválido")
    return c

  @field_validator("nome")
  @classmethod
  def valida_nome(cls, n: str) -> str:
    if not VALID_NOME_REGEX.match(n):
      raise ValueError("Nome inválido")
    return n

  @field_validator("role", mode="before")
  @classmethod
  def validate_role(cls, v: int | str | Role) -> Role:
        op = {
            int: lambda x: Role(x),
            str: lambda x: Role[x.strip().capitalize()],
            Role: lambda x: x
            }
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Função inválida, insira uma dessas funções: {', '.join([x.name for x in Role])}"
            )

  @field_serializer("id", when_used="json")
  def serialize_id(self, id: UUID4) -> str:
      return str(id)

Criação dos endpoints de requisição HTT.

In [50]:
app = FastAPI()

@app.get("/usuarios")
def get_usuario() -> list[Usuario]:
    return list(DB_USUARIOS.values())

@app.post("/usuarios")
def criar_usuario(usuario: Usuario) -> Usuario:
    DB_USUARIOS[usuario.id] = usuario
    return usuario

@app.get("/usuarios/{user_id}")
def get_usuario_por_id(user_id: UUID):
    usuario = DB_USUARIOS.get(user_id)

    if not usuario:
        return JSONResponse(status_code=404, content={"message": "Usuario nao encontrado"})

    return usuario.model_dump(mode="json")

Implementação do código responsável pelo controle das chamadas das classes e da API. Nesse caso foi criado um dicionário de teste para validar se está tudo certo. Foi testado se a serialização, validação e os endpoints estão funcionando de acordo.

In [58]:
DB_USUARIOS: dict[UUID, Usuario] = {}

def main() -> None:
  data = {
      "nome": "Arjan",
      "email": "example@arjancodes.com",
      "cpf": "123.456.789-12",
      "role": "Espectador",
  }

  bad_data = {
      "nome": "Arjan",
      "email": "example"
  }

  try:
    user = Usuario(**bad_data)
  except ValueError as e:
    print(e)
    print()

  user = Usuario(**data)
  print(user)


  #Serialização padrão (dict)
  print(user.model_dump())

  #Serialização json
  print(user.model_dump(mode="json"))

  #Teste dos endpoints
  with TestClient(app) as client:
    response = client.post("/usuarios", json=data)
    user_id = response.json()["id"]

    created_user = response.json()
    print("POST /usuarios:", created_user)


    response = client.get("/usuarios")
    print("GET /usuarios:", response.json())

    response = client.get(f"/usuarios/{user_id}")
    print("GET /usuarios/{id}:", response.json())

if __name__ == "__main__":
    main()

3 validation errors for Usuario
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='example', input_type=str]
cpf
  Field required [type=missing, input_value={'nome': 'Arjan', 'email': 'example'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
role
  Field required [type=missing, input_value={'nome': 'Arjan', 'email': 'example'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

id=UUID('ffe82d69-665b-4d4d-a7ee-692b47614edf') nome='Arjan' email='example@arjancodes.com' cpf=SecretStr('**********') role=<Role.Espectador: 1>
{'id': UUID('ffe82d69-665b-4d4d-a7ee-692b47614edf'), 'nome': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Espectador: 1>}
{'id': 'ffe82d69-665b-4d4d-a7ee-692b47614edf', 'nome': 'Arjan', 'email': 'example@arjancodes.com', 'role': 1}
POST /usuarios: {'id': 'e36140f3-f823-4ca6-8da7-1347623b622d', 'nome': 'A